In [15]:
import os
import random
import pickle
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
from gensim.models import Word2Vec
import lightgbm as lgb

import numpy as np
def get_data(path):
    df = pd.read_csv(path, encoding='gbk')

    train_y_label = df.iloc[:, -1]
    train_x_text = df.iloc[:, -9:-1]

    train_y_cost = df.loc[:, 'SUB_AMT']

    df = df.drop('SUB_AMT', axis=1)
    # df = df.drop('TOTAL_RECEIPT_AMT', axis=1)
    df.drop(df.columns[list(range(-9, 0, 1))], axis=1, inplace=True)
    train_x_figure = df

    return train_x_figure.values.tolist(), train_x_text.values, train_y_label.values.tolist(), train_y_cost.values.tolist()
def cat_word_2_vec(all_x_figure,texts):
    num, co_num = texts.shape

    for i in range(co_num):
        lis = []
        for j in range(num):
            lis.append([str(texts[j][i])])
        unique_list = list(set(map(tuple, lis)))
        lis = [list(item) for item in unique_list]

        if os.path.exists(f'lgb_model/w2v_{i}.npy'):
            with open(f'lgb_model/w2v_{i}.npy','rb') as f :
                word_2_vec=pickle.load(f)
        else:
            word_2_vec = Word2Vec(sentences=lis, vector_size=5, window=30, min_count=1)

            with open(f'lgb_model/w2v_{i}.npy','wb') as f :
                pickle.dump(word_2_vec,f)
        for j in range(len(all_x_figure)):
            all_x_figure[j] +=word_2_vec.wv[str(texts[j][i])].tolist()
    return all_x_figure
def get_trains_data(train_x_figure,train_y_label,rate=6):
    trains_data = [(train_x_figure, train_y_label)]
    return trains_data
def seed_everything(seed):
    np.random.seed(seed)
    random.seed(seed)
if __name__ == '__main__':
    seed_everything(3407)
    all_x_figure, all_x_text, all_y_label, all_y_cost = get_data('df_data_premodel_final.csv')
    all_x_figure = cat_word_2_vec(all_x_figure,all_x_text)
    X, y =  all_x_figure,all_y_label# 特征和标签
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

    trains_data = get_trains_data(X_train,y_train)

    d_k = len(trains_data)

    params = {
        'objective': 'multiclass',
        'num_class': 2,
        'max_depth': 20,
        'learning_rate': 0.1,
        'num_leaves': 256
    }

    all_pre = np.array([0.0]*len(X_test))
    for k in range(d_k):
        X_train,y_train = trains_data[k]
        dtrain = lgb.Dataset(np.array(X_train), label=np.array(y_train))
        dtest = lgb.Dataset(np.array(X_test), label=np.array(y_test))
        if os.path.exists(f"lgb_model/model_{k}.dat"):
            model = pickle.load(open(f"lgb_model/model_{k}.dat", "rb"))
        else:
            # model = xgb.train(params, dtrain, num_boost_round=40)
            model = lgb.train(params, dtrain, num_boost_round=150)
            pickle.dump(model, open(f"lgb_model/model_{k}.dat", "wb"))

        y_pred = model.predict(X_test)
        y_pred = np.argmax(y_pred, axis=1)#结果
        accuracy,f1,recall,prec = accuracy_score(y_test, y_pred),f1_score(y_test, y_pred),recall_score(y_test, y_pred),precision_score(y_test, y_pred)
        print(f'accuracy_score:{accuracy} f1_score:{f1} recall_score{recall} precision_score{prec}')




accuracy_score:0.9605263157894737 f1_score:0.853907134767837 recall_score0.7887029288702929 precision_score0.9308641975308642
